# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.linear_model import LinearRegression 
from sklearn.preprocessing import StandardScaler
from numpy.random import RandomState

In [2]:
RANDOM_STATE = RandomState(12345) 

## Загрузка и подготовка данных

In [3]:
try :
    data_1 = pd.read_csv('geo_data_0.csv')
    data_2 = pd.read_csv('geo_data_1.csv')
    data_3 = pd.read_csv('geo_data_2.csv')
except:
    data_1 = pd.read_csv('/datasets/geo_data_0.csv')
    data_2 = pd.read_csv('/datasets/geo_data_1.csv')
    data_3 = pd.read_csv('/datasets/geo_data_2.csv')

In [4]:
data_1.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [5]:
data_2.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [6]:
data_3.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


### Проверим данные на дубликаты.

In [7]:
data_1[data_1.duplicated('id')]['id']

7530     HZww2
41724    bxg6G
51970    A5aEY
63593    QcMuo
66136    74z30
69163    AGS9W
75715    Tdehs
90815    fiKDv
92341    TtcGQ
97785    bsk9y
Name: id, dtype: object

In [8]:
data_2[data_2.duplicated('id')]['id']

41906    LHZR0
82178    bfPNe
82873    wt4Uk
84461    5ltQ6
Name: id, dtype: object

In [9]:
data_3[data_3.duplicated('id')]['id']

43233    xCHr8
49564    VF7Jo
55967    KUPhW
95090    Vcm5J
Name: id, dtype: object

Изучив дубликаты, можно предположить, что дубликаты могли быть возникнуты по причине возможного пересчета данных. Для решения этой проблемы возьмем среднее значение по каждому признаку и тем самым избавимся от дубликатов.

Напишем функцию, которая преобразует таблицы, убрав дубликаты. Для этого создадим доп таблицу, где для дублирующихся скважен посчитаем среднее значения их признаков. Удалим из основной таблицы дубликаты, а в оставшихся столбцах поменяем значения на средние.  

In [10]:
def new_table(table):
    id_dup = table[table.duplicated('id')]['id']
    dup_mean=table.query("id in @id_dup").pivot_table(index='id', aggfunc='mean')
    table_drop=table.drop_duplicates(subset=['id']).reset_index(drop=True)
    for elem_id in id_dup:
        table_drop.loc[table_drop['id']==elem_id, ('f0', 'f1', 'f2', 'product')] = dup_mean.loc[[elem_id]].values.tolist()
    return table_drop

In [11]:
data_1=new_table(data_1)
data_2=new_table(data_2)
data_3=new_table(data_3)

In [12]:
print('Дубликатов в data_1:',data_1.duplicated('id').sum())
print('Дубликатов в data_2:',data_2.duplicated('id').sum())
print('Дубликатов в data_3:',data_3.duplicated('id').sum())

Дубликатов в data_1: 0
Дубликатов в data_2: 0
Дубликатов в data_3: 0


Далее проверим есть ли пропуски в наших данных.

In [13]:
print('Пропусков в data_1','\n', data_1.isna().sum(), sep='')
print('Пропусков в data_2','\n', data_2.isna().sum(), sep='')
print('Пропусков в data_3','\n', data_3.isna().sum(), sep='')

Пропусков в data_1
id         0
f0         0
f1         0
f2         0
product    0
dtype: int64
Пропусков в data_2
id         0
f0         0
f1         0
f2         0
product    0
dtype: int64
Пропусков в data_3
id         0
f0         0
f1         0
f2         0
product    0
dtype: int64


Раз мы избавились от дубликатов, то каждая строка является уникальной, поэтмоу мы можем удалить столбец id.

In [14]:
data_1=data_1[['f0', 'f1', 'f2', 'product']]
data_2=data_2[['f0', 'f1', 'f2', 'product']]
data_3=data_3[['f0', 'f1', 'f2', 'product']]

### Разобьем данные на обучающие и валидационные

In [15]:
features_1 = data_1.drop(columns = ['product'],axis = 1)
target_1 = data_1['product']
features_2 = data_2.drop(columns = ['product'],axis = 1)
target_2 = data_2['product']
features_3 = data_3.drop(columns = ['product'],axis = 1)
target_3 = data_3['product']

In [16]:
features_train_1, features_valid_1, target_train_1, target_valid_1 = train_test_split(
    features_1, target_1, test_size=0.25, random_state=RANDOM_STATE)
features_train_2, features_valid_2, target_train_2, target_valid_2 = train_test_split(
    features_2, target_2, test_size=0.25, random_state=RANDOM_STATE)
features_train_3, features_valid_3, target_train_3, target_valid_3 = train_test_split(
    features_3, target_3, test_size=0.25, random_state=RANDOM_STATE)

Также нормализуем данные

In [17]:
def data_scaler(f_t, f_v):
    t_index = f_t.index
    v_index = f_v.index
    scaler = StandardScaler()
    scaler.fit(f_t)
    return pd.DataFrame(data=scaler.transform(f_t), index=t_index), pd.DataFrame(data=scaler.transform(f_v), index=v_index)

In [18]:
features_train_1, features_valid_1 = data_scaler(features_train_1, features_valid_1)
features_train_2, features_valid_2 = data_scaler(features_train_2, features_valid_2)
features_train_3, features_valid_3 = data_scaler(features_train_3, features_valid_3)

## Обучение и проверка модели

In [19]:
def model_param(f_train, t_train, f_valid, t_valid):
    model = LinearRegression()
    model.fit(f_train, t_train)
    predictions = model.predict(f_valid)
    rmse = (mean_squared_error(t_valid, predictions))**(0.5)
    print('RMSE:', rmse)
    print('Средний запас предсказанного сырья:', predictions.mean())
    return pd.Series(predictions, index=f_valid.index)

In [20]:
predictions_1 = model_param(features_train_1, target_train_1, features_valid_1, target_valid_1)

RMSE: 37.852213368277745
Средний запас предсказанного сырья: 92.79006774683418


In [21]:
predictions_2 = model_param(features_train_2, target_train_2, features_valid_2, target_valid_2)

RMSE: 0.8947792873072147
Средний запас предсказанного сырья: 68.95699542947811


In [22]:
predictions_3 = model_param(features_train_3, target_train_3, features_valid_3, target_valid_3)

RMSE: 40.041547529175915
Средний запас предсказанного сырья: 95.08069522090472


### Вывод



Судя по полученным данным, наименьший RMSE имеет модель по данным второго региона, а средний запас предсказанного сырья в первом и третьем регионах приблизительно одинаковые, во втором меньше на несколько десятков.

## Подготовка к расчёту прибыли

In [23]:
BUDGET = 10 * 1e9 
PRICE_PRODUCT = 450000
COUNT_POINTS = 500 
COUNT_BEST_POINTS = 200 
LOSS_LESS = 0.025

Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины.

In [24]:
budget_point = BUDGET / COUNT_BEST_POINTS
enough_product = budget_point / PRICE_PRODUCT
print('Достаточный объём сырья для безубыточной разработки новой скважины:', enough_product)

Достаточный объём сырья для безубыточной разработки новой скважины: 111.11111111111111


Средний запас в каждом регионе не достигает достаточного объема для безубыточной разработки, получается каждый регион будет нести какие-то убытки.

Зададим функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели.

In [25]:
def get_revenue(predict, target):
        points = predict.sample(n=COUNT_POINTS, replace=True, random_state=RANDOM_STATE)
        best_points = points.sort_values(ascending = False)[:COUNT_BEST_POINTS]
        product_volume = target[best_points.index].sum()
        revenue = PRICE_PRODUCT * product_volume - BUDGET
        return revenue, product_volume

In [26]:
vol_reven=get_revenue(predictions_1, target_valid_1)
print(f'Объём сырья {COUNT_BEST_POINTS} скважин с максимальными значениями предсказаний = {vol_reven[1]:.2f} тысяч баррелей\nПрибыль = {(vol_reven[0]/1e6):.2f} миллионов рублей')

Объём сырья 200 скважин с максимальными значениями предсказаний = 23298.67 тысяч баррелей
Прибыль = 484.40 миллионов рублей


In [27]:
vol_reven=get_revenue(predictions_2, target_valid_2)
print(f'Объём сырья {COUNT_BEST_POINTS} скважин с максимальными значениями предсказаний = {vol_reven[1]:.2f} тысяч баррелей\nПрибыль = {(vol_reven[0]/1e6):.2f} миллионов рублей')

Объём сырья 200 скважин с максимальными значениями предсказаний = 22377.01 тысяч баррелей
Прибыль = 69.66 миллионов рублей


In [28]:
vol_reven=get_revenue(predictions_3, target_valid_3)
print(f'Объём сырья {COUNT_BEST_POINTS} скважин с максимальными значениями предсказаний = {vol_reven[1]:.2f} тысяч баррелей\nПрибыль = {(vol_reven[0]/1e6):.2f} миллионов рублей')

Объём сырья 200 скважин с максимальными значениями предсказаний = 23708.74 тысяч баррелей
Прибыль = 668.93 миллионов рублей


## Расчёт прибыли и рисков 

Посчитаем риски и прибыль для каждого региона:

In [29]:
def revenue_bootstrap(predict, target):
    values=[]
    for i in range(1000):
        values.append(get_revenue(predict, target)[0])
    values = pd.Series(values)
    revenue_mean = values.mean()
    interv_lower = values.quantile((1 - 0.95) / 2)
    interv_upper = values.quantile((1 + 0.95) / 2)
    loss_probability = (values < 0).sum() / 1000 
    print('Риск убытков =', loss_probability)
    print(f'Средняя прибыль = {revenue_mean/1e6:.2f} миллионов рублей')
    print(f'95% доверительный интервал = ({interv_lower/1e6:.2f}, {interv_upper/1e6:.2f}), интервал = {(interv_upper - interv_lower)/1e6:.2f}')


In [30]:
revenue_bootstrap(predictions_1, target_valid_1)

Риск убытков = 0.074
Средняя прибыль = 389.21 миллионов рублей
95% доверительный интервал = (-111.84, 894.00), интервал = 1005.84


In [31]:
revenue_bootstrap(predictions_2, target_valid_2) 

Риск убытков = 0.013
Средняя прибыль = 439.08 миллионов рублей
95% доверительный интервал = (33.06, 850.15), интервал = 817.08


In [32]:
revenue_bootstrap(predictions_3, target_valid_3)

Риск убытков = 0.046
Средняя прибыль = 449.81 миллионов рублей
95% доверительный интервал = (-62.39, 943.66), интервал = 1006.05


В итоге лучшим регионом для разработки скважин будет регион №2, так как он имеет наименьший риск убытков, с учетом того, что максимально допустимая вероятность возникновения - 0.025. Так же стоит заметить, что второй регион имеет самую высокую среднюю прибыль, а доверительный интервал наоборот - принимает минимальное значение из трех.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: модели обучены и проверены
    - [x]  Данные корректно разбиты на обучающую и валидационную выборки
    - [x]  Модели обучены, предсказания сделаны
    - [x]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [x]  На экране напечатаны результаты
    - [x]  Сделаны выводы
- [x]  Выполнен шаг 3: проведена подготовка к расчёту прибыли
    - [x]  Для всех ключевых значений созданы константы Python
    - [x]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [x]  По предыдущему пункту сделаны выводы
    - [x]  Написана функция расчёта прибыли
- [x]  Выполнен шаг 4: посчитаны риски и прибыль
    - [x]  Проведена процедура *Bootstrap*
    - [x]  Все параметры бутстрепа соответствуют условию
    - [x]  Найдены все нужные величины
    - [x]  Предложен регион для разработки месторождения
    - [x]  Выбор региона обоснован